In [1]:
import numpy as np
#lit un fichier avec des couple <utilisateur>:<temps d'infection> séparé par des :
#renvoi un tableu des episodes composé de couple <utilisateur>:<temps d'infection>
#renvoi également la liste des utilisateurs
def read_file(file):
    episode = []
    liste_utilisateurs = []
    with open(file) as f:
        for line in f.readlines():
            affecion_episode = []
            liste_couple = line.split(';')
            for couple in liste_couple[:-1]:
                couple = couple.split(':')
                affecion_episode.append([int(couple[0]),float(couple[1])])
                if(int(couple[0]) not in liste_utilisateurs):
                    liste_utilisateurs.append(int(couple[0]))
            episode.append(affecion_episode)
    return episode, liste_utilisateurs

train, liste_utilisateurs = read_file('cascades_train.txt')

test,_ = read_file("cascades_test.txt")

#liste_utilisateurs = sorted(liste_utilisateurs)

#print(liste_utilisateurs)
#print(len(liste_utilisateurs))

#renvoie la liste triée selon le 2éme element
def sort_time(liste_episodes):
    sort_liste = []
    for episode in liste_episodes:
        sort_liste.append(sorted(episode, key = lambda x: float(x[1])))
    return sort_liste

train = sort_time(train)


In [2]:
import json

#ne marche pas ... trop long 
def create_D_plus(liste_episodes):
    D_plus = {}
    for num,episode in enumerate(liste_episodes):
        num_episode = [num]
        for i,couple in enumerate(episode):
            u = couple[0]
            time = couple[1]
            if(len(episode[i+1:])>=1):
                for couple_infect in episode[i+1:]:
                    u_infect = couple_infect[0]
                    time_infect = couple_infect[1]
                    if time < time_infect:
                        key = str(str(u) + ',' + str(u_infect))
                        if key in D_plus:
                            num_episode += D_plus[key]
                        D_plus[key] = num_episode
                        #remet num au numéro d'episode sinon cumulation de tout les épisodes dans le meme D => marche pas
                        num_episode = [num]
#                         print(D_plus[key])
        #print(D_plus)
#     print(json.dumps(D_plus, indent = 1))
    return D_plus
                
# create_D_plus(train)

In [3]:
#compte 
def create_D_moins(liste_episodes,liste_utilisateurs):
    D_moins = np.zeros((len(liste_utilisateurs),len(liste_utilisateurs)))
#     print(D_moins[0][0])
#     print(gr)
    for episode in liste_episodes:
        episode = np.array(episode)
#         print('episode \n' ,episode)
        utilisateurs_par_episode = episode[:,0]
#         print(utilisateurs_par_episode)
        for couple in episode:
            u = int(couple[0])
#             print('utilisateur \n',u)
            for utilisateur in liste_utilisateurs:
#                 print('utilisateur \n', utilisateur)
#                 print(episode[:,0])
                if utilisateur not in utilisateurs_par_episode:
                    D_moins[u][utilisateur] += 1.0
    return D_moins


d_plus = create_D_plus(train)
d_moins = create_D_moins(train,liste_utilisateurs)
# create_D_moins(train,liste_utilisateurs)

In [33]:

#renvoie la liste des preds de u
#ceux dont le temps est inferieurs à u
def pred_u(episode,u):
#     print(episode)
    liste_preds = []
    for couple in episode:
        if couple[0] == u:
            time_u = couple[1]
            return [ c[0]  for c in liste_preds if c[1]<time_u]
        else :
            liste_preds.append(couple)
    return liste_preds

# print(pred_u(train[0],12))

def p_u(episode,u,theta):
    preds = pred_u(episode,u)
    if len(preds) == 0:
        return 1,1
    res = 1
    #produit de 1 - theta sur les preds de u
    for i in preds:
        res *= 1-theta[i][u]
    return 1-res,res


# theta = np.random.rand(100,100)
# print(p_u(train[0],56,theta))
# print(p_u(train[0],42,theta))

def somme_theta_d_plus(u,v,theta,d_plus,p):
    somme = 0
    key = str(u) + "," + str(v)
    #print(d_plus[key])
    for i in d_plus[key]:
        somme += theta[u][v] / p
    return somme
    
# somme_theta_d_plus(0,1,theta,create_D_plus(train))
def train_(liste_episodes,liste_utilisateurs):
    
    theta = np.random.rand(len(liste_utilisateurs),len(liste_utilisateurs))
    for i,episode in enumerate(liste_episodes):
        liste_util_episode = np.array(episode,dtype=int)[:,0]
       
        for i,u in enumerate(liste_util_episode):
            for v in liste_util_episode[:i:]:
#                 print(u,v)
                key = str(u) + "," + str(v)
                d_plus_u_v = d_plus[key]
                if len(d_plus_u_v) > 0:
                    
                    p_infection,res = p_u(episode,u,theta)
    #                 print(theta[u][v])
                    quotient = d_moins[u][v] + len(d_plus[key])
                    print(episode)
                    print(gr)
                    if p_infection == 0:
                        print(len(d_plus_u_v))
                        print(len(episode))
                        print(i,episode)
                        print(liste_util_episode)
                        print(u)
                        print(res)
                        print(gr)
                    theta[u][v] = somme_theta_d_plus(u,v,theta,d_plus,p_infection) / quotient
#                 print(theta[u][v])
#                 print(gr)
#                 print(u,episode)

    return theta

train_(train,liste_utilisateurs)

[[56, 1.0], [26, 1.0], [42, 2.0], [41, 2.0], [93, 2.0], [89, 3.0], [8, 3.0], [3, 4.0], [43, 4.0], [73, 4.0], [84, 5.0], [61, 5.0], [98, 5.0], [86, 5.0], [48, 6.0], [47, 6.0], [96, 7.0], [50, 7.0], [55, 7.0], [20, 7.0], [12, 7.0], [5, 8.0], [52, 8.0], [54, 9.0], [34, 9.0], [4, 10.0], [82, 11.0]]


NameError: global name 'gr' is not defined

In [26]:
print(train[2])

[[81, 1.0], [35, 1.0], [4, 1.0], [94, 2.0], [64, 3.0], [16, 3.0], [46, 3.0], [17, 4.0], [39, 4.0], [60, 4.0], [7, 4.0], [2, 4.0], [23, 4.0], [58, 4.0], [47, 4.0], [29, 5.0], [48, 5.0], [71, 5.0], [8, 5.0], [25, 5.0], [72, 5.0], [19, 6.0], [68, 6.0], [3, 6.0], [34, 6.0], [67, 7.0], [9, 7.0], [73, 7.0], [89, 8.0]]
